# KNN with HOG

Needed packages

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from preprocessing.hog import Flatten

Adding our own packages to the system path so that they can be found.

In [39]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Loading Metadata

We load the hog metadata file in order to find the paths to the training and testing data.

In [42]:
hog_metadata_path = '../../data/hog/metadata.csv'
hog_metadata = pd.read_csv(hog_metadata_path, index_col=0)
hog_metadata.channels = hog_metadata.channels.astype(str)

## Creating the KNN Pipeline

In [67]:
pipeline = Pipeline([('flatten', Flatten()), ('knn', KNeighborsClassifier(n_neighbors=7))])

## Getting the data, fitting, scoring

We go through all of the channels, and fetch the train and test data associated with each. Then we fit and score the pipeline.

In [73]:
for channel in hog_metadata.channels.unique():
    train_metadata = hog_metadata[(hog_metadata.type == 'train') & (hog_metadata.channels == channel)]
    train = np.load(f'../../{train_metadata .data_path.values[0]}')
    train_label = np.load(f'../../{train_metadata.label_path.values[0]}')

    test_metadata = hog_metadata[(hog_metadata.type == 'test') & (hog_metadata.channels == channel)]
    test = np.load(f'../../{test_metadata.data_path.values[0]}')
    test_label = np.load(f'../../{test_metadata.label_path.values[0]}')
    
    pipeline.fit(train, train_label)
    print(pipeline.score(test, test_label))

0.5363636363636364
0.4818181818181818
0.5
